In [1]:
%matplotlib inline

# generic
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# custom
from icap.database.icapdatabase import ICapDatabase
from icap.results.results import Results

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

if conn.__class__.__name__ != 'Connection':
    raise Exception('No Connection')

In [3]:
from icap.coned.coned import CONEDRecipe

In [ ]:
cr = CONEDRecipe(conn, Results).run_all()

In [ ]:
prems_to_inspect = [
    ('494201000700002','low_variance_1.1%'),
    ('544582782020002','zero_icap'),
    ('677606742000009','zero_icap'),
    ('233774551600009','zero_icap'),
    ('700536121000000','cons_not_dmd')
]

In [ ]:
from icap.coned.coned import CONEDMonthly

In [ ]:
cm = CONEDMonthly(conn, premise='644154216000007', show_lst=True)

In [ ]:
cm.compute_icap().drop_duplicates()

In [ ]:
mcd_out = cm.compute_mcd()

In [ ]:
cm.monthly

In [ ]:
prem, year, rc, strata, zone, stratum, tod, startdate, enddate, *_, meter_regex = cm.monthly.reset_index().ix[0]
rc, stratum = map(int, [rc, stratum])

billcycle = cm.temp_var.ix[startdate: enddate]
print('Days in billcycle: {}'.format(billcycle.shape[0]))

# merge the billcycle with loadshapetemp adjustment table
local_lst = pd.merge(billcycle, cm.lst,
                    left_index=True, right_index=True,
                    on=['Max', 'DayOfWeek'])

print('LST shape after BillCycle Merge: {}'.format(local_lst.shape[0]))
strat_mask = (local_lst['STRAT L BOUND'] <= stratum) & (local_lst['STRAT U BOUND'] > stratum)
sc_mask = local_lst['SC'] == cm.rc_map.ix[rc]['Map']
tod_mask = local_lst['STRATA'].str.contains(meter_regex)

local_lst[strat_mask & sc_mask & tod_mask].sort_index()


In [ ]:
cm.compute_mcd()

In [ ]:
cm.monthly

In [ ]:
cr.compare_.head()

In [ ]:
cr.write_comparison_to_csv()
cr.analyze_comparison(write_to_excel=True)

In [ ]:
df = cr.compare_.dropna(subset=['RecipeVariance'])
full_shape = df.shape[0]

var_mask = df.RecipeVariance <= 2.0
kw_mask = abs(df.RecipeICap - df.HistoricalICap) <= 1.0
mask = (var_mask == 1) | (kw_mask == 1)


filtered = df.ix[mask].shape[0]

print("full shape ", full_shape)
print("filtered shape ", filtered)
print("ratio: ", filtered / full_shape)

In [ ]:
?np.round

In [ ]:
def meter_year_analysis(g: pd.core.groupby.DataFrameGroupBy):
    
    
    df = g.dropna(subset=["RecipeVariance"])
    
    var_mask = df.RecipeVariance <= 2.0
    kw_mask = abs(df.RecipeICap - df.HistoricalICap) <= 1.0
    mask = (var_mask == 1) | (kw_mask == 1)

    df_filtered = df.ix[mask]
    
    return df.shape[0], df_filtered.shape[0], np.round(df_filtered.shape[0] / df.shape[0], decimals=4)
    
    

In [ ]:
grp = df.groupby(['MeterType', 'Year'])
grp.apply(meter_year_analysis)

In [ ]:
from icap.coned.coned import CONEDMonthly
prem = CONEDMonthly(conn, premise='443023022420001')
prem.compute_mcd()
prem.compute_icap()

In [ ]:
prem.monthly

In [ ]:
prem.compute_mcd()
prem.tmp['443023022420001']

# Coned Monthly

In [ ]:
from icap.coned.coned import CONEDMonthly

In [ ]:
cm = CONEDMonthly(conn, premise='233774551600009')

In [ ]:
cm.compute_mcd()

In [ ]:
cm.compute_icap()

In [ ]:
cm.tmp[('233774551600009', 'lst')]
#cm.tmp[('233774551600009', 'bill_cycle')].shape[0]

In [ ]:
writer = pd.ExcelWriter('494201000700002_calculation.xlsx')

In [ ]:
cm.tmp['233774551600009']

In [ ]:
cm.tmp['494201000700002'].to_excel(writer, 'load_profile')
cm.temp_var.to_excel(writer, 'temp_var')
writer.save()
writer.close()

In [ ]:
cm.monthly.head()

In [ ]:
from icap.coned.coned import CONEDInterval
ci = CONEDInterval(conn)

In [ ]:
ci.compute_mcd()

In [ ]:
ci.hourly.ix['494122335500026']

In [ ]:
cm.compute_mcd()

In [ ]:
r = cm.compute_icap()

In [ ]:
# defaulting to lower bound
r[r['PremiseId'] == '494121202700008'].drop_duplicates()

In [ ]:
bill_start, bill_end = cm.monthly.xs(('233774551600009', 2015))[['StartDate', 'EndDate']].values

cm.lst.ix[bill_start: bill_end];
print(bill_start,' ', bill_end)

In [ ]:
local_lst = pd.merge( cm.temp_var.ix[bill_start : bill_end], cm.lst, 
                    left_index=True, right_index=True, on=['Max', 'DayOfWeek'])

# filter for stratum condition
stratum = float(cm.monthly.xs(('233774551600009', 2015))['Stratum'])
stratum_lower_mask = local_lst['STRAT L BOUND'] <= stratum
stratum_upper_mask = local_lst['STRAT U BOUND'] > stratum
straum_mask = (stratum_lower_mask == 1 ) & (stratum_upper_mask == 1)
print(stratum)

local_filtered = local_lst.ix[straum_mask]

#meter_regex = cm.monthly.xs(('494121202700008', 2015))['MeterRegex']

tod_mask = local_filtered['STRATA'].str.contains('[T]')
sc_mask = (local_filtered['SC'] == cm.rc_map.ix[116]['Map'])
mask = (tod_mask == 1) & (sc_mask == 1)

final_lst = local_filtered.ix[mask]

final_lst.shape

In [ ]:
print(local_filtered.shape)
print(local_filtered[local_filtered['STRATA'].str.contains('[T]')].shape)

local_filtered[local_filtered['SC'] == cm.rc_map.ix[116]['Map']]

In [ ]:
final_lst.sort_values(by='DayOfWeek')

In [ ]:
final_lst.shape

In [ ]:
cm.temp_var.ix[bill_start:bill_end].shape

## Temp Variant Testing

In [ ]:
temp_station_query = """select
                RTrim(StationCode) as StationCode,
                ObservedDate, --Hour,
                Temperature, WetBulbTemperature
            from CONED_NYWeatherData
            order by
                ObservedDate"""

ts = pd.read_sql(temp_station_query, conn)

In [ ]:
ts.head();

In [ ]:
"""
Hourly Average:
    WetBulbTemperature = WBT; Temperature = T;
    
    for hour in ObservedDate:
        hourly_avg[i] = 0.25 * (KNYC_WBT + KNYC_T + KLGA_WBT + KLGA_T)

"""
hourly_avg = pd.pivot_table(ts, index='ObservedDate', 
               columns='StationCode', 
               values=['Temperature', 'WetBulbTemperature'],
                    ).mean(1)

hourly_avg['2015-05-01 09:00:00':'2015-05-01 21:00:00']

In [ ]:
pd.pivot_table(ts, index='ObservedDate',
              columns='StationCode',
              values=['Temperature', 'WetBulbTemperature'])['2015-05-01 09:00:00':'2015-05-01 21:00:00']

In [ ]:
"""
Rolling Average:
    1. Group hourly average into days
    2. Rolling mean over 3 hour window
    3. Take maximum average per day
"""
daily_max_avg = hourly_avg.groupby(pd.TimeGrouper('D')
                  ).rolling(window=3).mean().max(level=0)

In [ ]:
"""
Rolling Weighted Sum:
    The weighted sum is applyed to 3 day rolling window.
    The current day weight is 70%, day-1 is 20%, day-2 is 10%.
    
    weights = [0.1, 0.2, 0.7]
    day[i-2], day[i-1], day[i] = weights 


"""
# helper function to compute weighted sum
def f(w):
    def g(x):
        return (w * x).sum()
    return g

# Weights
wts = np.array([.1, .2, .7])

# Apply rolling weighted summation
daily_max_avg.rolling(window=3).apply(f(wts));



daily_max_avg['2015-05-01']

In [ ]:
# CONVERSIONS
# convert hour into timedelta
# increment `ObservedDate` by correspoding Timedelta
# drop the `Hour` columns
ts['Hour'] = ts['Hour'].apply(lambda x: pd.Timedelta(hours=x))
ts['ObservedDate'] = ts['ObservedDate'] + ts['Hour']
td = ts.drop('Hour', axis=1)

In [ ]:
# update index
#td.set_index('ObservedDate', inplace=True)
ts.set_index('ObservedDate', inplace=True)
ts.drop('Hour', axis=1, inplace=True)

In [ ]:
ts.head()

In [ ]:
# AGGREGATION
# Station_i : sum = temperature + wetbulbtemperature

#td['RowSum'] = td.sum(axis=1)
ts['RowSum'] = ts.sum(axis=1)

In [ ]:
knyc = td[td['StationCode'] == 'KNYC']
klga = td[td['StationCode'] == 'KLGA']

In [ ]:
knyc.head()

In [ ]:
klga.head()

In [ ]:
ts.drop(['Temperature', 'WetBulbTemperature'], axis=1
       ).pivot(columns='StationCode', values='RowSum')

In [ ]:
# drop columns aggregated in `RowSum`
# pivot stations into columns
# avg: (Station_i:RowSum) + (Station_j:RowSum) * (1/4)
hr_avg = td.drop(['Temperature', 'WetBulbTemperature'], axis=1
                 ).pivot(columns='StationCode', values='RowSum'
                         ).apply(lambda row: row.sum() * .25, axis=1)

--

In [ ]:
from icap.coned.coned import CONEDRecipe
c = CONEDRecipe(conn=conn, results=Results).run_all()

In [ ]:
c.write_comparison_to_csv()
c.analyze_comparison(write_to_excel=True)

## CONED Interval

In [ ]:
%%time
from icap.coned.coned import CONEDInterval
c = CONEDInterval(conn)

In [ ]:
c.hourly.head().columns

In [ ]:
from icap.results.results import Results
r = Results(conn,c.compute_icap())


In [ ]:
r.analyze_comparison()

In [ ]:
hourly_cp = c.get_hourly_cp()

In [ ]:
??c.meter_logic

In [ ]:
hourly_rec = c.get_hourly()

In [ ]:
hourly_query = """
    select
        h.PremiseId,
        p.RateClass, ce.[Service Classification],
        ce.[Zone Code] as ZoneCode,
        ce.[Stratum Variable] as Stratum,
        ce.[Time of Day Code] as TOD,
        Year(m.EndDate) as Year,
        DateAdd(day, 0,  m.StartDate) as StartDate,
        m.EndDate,
        m.Usage as BilledUsage,
        m.Demand as BilledDemand,
        Round(Sum(h.Usage), 0) as CPHourUsage,
        'INT' as MeterType,
        iif(Abs((m.Usage-Sum(h.Usage))/m.Usage)<=0.04, 1, 0) as VarTest
    from [HourlyUsage]  h
    inner join [MonthlyUsage]  m
        on m.PremiseId = h.PremiseID
        and m.UtilityID = h.UtilityId
    inner join CoincidentPeak as cp
        on cp.UtilityId = h.UtilityId
        and Year(cp.CPDate) = Year(m.EndDate)
        and (cp.CPDate between m.StartDate and m.EndDate)
        and (h.UsageDate between m.StartDate and m.EndDate)
    inner join Premise as p
        on p.PremiseId = h.PremiseId
    inner join ConED as ce
        on CAST(ce.[Account Number] as varchar) = h.PremiseId
    where
        h.UtilityId = 'CONED'
        and h.HourEnding between 1 and 24
    group by
        h.PremiseId, MeterType,
        p.RateClass, ce.[Service Classification],
        ce.[Zone Code], ce.[Stratum Variable], ce.[Time of Day Code],
        Year(m.EndDate),
        m.StartDate, m.EndDate,
        m.Usage,
        m.Demand
    having
        Count(h.Usage) = (DateDiff(hour, m.StartDate, m.EndDate) + 24)
        """
# obtain data; set defaults; converions
df = pd.read_sql(hourly_query,conn)

In [ ]:
df.head()

In [ ]:
df['MCD'] = np.NaN
df['NormUsage'] = np.NaN
df['TOD'] = df['TOD'].apply(lambda x: np.int(x))

In [ ]:
# create multi-index; sort
df.set_index(['PremiseId', 'Year'], inplace=True)
df.sort_index(inplace=True)

In [ ]:
df

In [ ]:
# determine meter type
df['MeterLogic'] = df.apply(self.meter_logic, axis=1)
df['MeterRegex'] = df['MeterLogic'].apply(tod_regex)

In [ ]:
r = Results(conn, c.compute_icap())

In [ ]:
r.analyze_comparison()

### Interval Varinace <= 4%

In [ ]:
# select MCD; Usage[CP_date, CP_hour]
tmp = pd.merge(c.varTrue.reset_index(), c.hourly_cp, 
        how='left',
        on=['PremiseId', 'Year'])

In [ ]:
tmp2 = pd.merge(tmp, c.util,
        how='left',
        left_on=['Year', 'ZoneCode'],
        right_on=['Year', 'Zone'])

In [ ]:
match_mask = tmp2['MeterLogic'] == tmp2['MeterType_y']
all_mask = tmp2['MeterType_y'] == 'ALL'
mask = (match_mask == 1) | (all_mask == 1)

tmp2['Factor'] = tmp2['Factor'].apply(lambda x: x + 1.0)


tmp2['MCD'] = tmp2['Usage']
tmp2.ix[mask].groupby(['PremiseId', 'Year', 'RateClass', 'MeterLogic']).apply(coned_icap)

In [ ]:
tmp2['Factor'] = tmp2['Factor'].apply(lambda x: x + 1.0)

In [ ]:
c.__dict__.keys()

In [ ]:
def coned_icap(g):
    mcd = g['Usage'].values
    stf = g[g['ParameterId'] == 'SubzoneTrueupFactor']['Factor'].values
    ftf = g[g['ParameterId'] == 'ForecastTrueupFactor']['Factor'].values
    try:
        icap = mcd[0] * stf[0] * ftf[0]
    except IndexError:
        icap = np.nan
    return icap

In [ ]:
tmp2.groupby(['PremiseId', 'Year', 'RateClass']).apply(coned_icap)

## CONED Monthly

In [ ]:
%%time
from icap.coned.coned import CONEDMonthly
c = CONEDMonthly(conn=conn)

In [ ]:
%%time
c.compute_mcd()

In [ ]:
icap = c.compute_icap()

In [ ]:
icap.head()

In [ ]:
r = Results(conn, icap)

In [ ]:
r.analyze_comparison(write_to_excel=True)

In [ ]:
results = r.compare_.copy()

In [ ]:
null_idx = results[pd.isnull(results['HistVar'])].index
valid_idx = results[results['HistVar'] <= 2.0].index
invalid_idx = results[results['HistVar'] > 2.0].index

In [ ]:
# assign values to outcomes on their index
results['Valid'] = ''
results.set_value(null_idx, 'Valid', 'NULL')
results.set_value(invalid_idx, 'Valid', 0)
results.set_value(valid_idx, 'Valid', 1)

In [ ]:
# aggregate and count
details = results.groupby(['MeterType', 'Year', 'RateClass',
                           'Strata', 'Valid']
                          )[['ICap', 'CapacityTagValue']].count()

In [ ]:
details

In [ ]:
comp = r.compare_.copy()
null_idx = comp[pd.isnull(comp['HistVar'])].index
valid_idx = comp[comp['HistVar'] <= 2.0].index
invalid_idx = comp[comp['HistVar'] > 2.0].index

In [ ]:
comp.ix[valid_idx].shape

In [ ]:
comp.ix[invalid_idx].shape

In [ ]:
r.analyze_comparison()

In [ ]:
def is_tod(rate_class):
    """Returns proper REGEX based on rate class. Meters can be either
    TIME OF DAY (if TODQ == 1 then 'Ta
') or 
    NOT TIME OF DAY (if TODQ == 0 then '[^T]')
    
    if c.rc_map.ix[rate_class]['TODQ']:
        return 'T'
    return '[^T]'
    """
    if meter_logic == 'VTOU':
        return 'T'
    return '[^T]'
        

In [ ]:
# Query the Load Shape Adjustment Table
lst_qry = "select * from CONED_LoadShapeTempAdj where Strata != ''"
lst = pd.read_sql(lst_qry, conn)

## OPTIMIZING THE LOAD PROFILE PROCESS
# 1) Merge the  Load Shape Adjustment Table with Temperature Variants on
#    the DAY[TYPE, OfWeek] columns. 
# 2) Then filter the table where [TEMP L BOUND] <= Max <= [TEMP U Bound]
tmp = pd.merge(lst, c.temp_var.reset_index(), left_on='DAYTYPE', right_on='DayOfWeek')
tmp = tmp[(tmp['TEMP L BOUND'] <= tmp['Max']) & (tmp['Max'] <= tmp['TEMP U BOUND'])]

In [ ]:
tmp.sort_values(by=['ObservedDate'])
tmp2 = tmp.set_index('ObservedDate')

In [ ]:
from datetime import datetime
from itertools import islice

"""When BilledDemand == 0, the MCD calculation chooses the zero value instead of
NormalizedUsage. Any location where a zero occurs is replaced with np.inf. This
ensures that when BilledDemand has a zero value, the NormalizedValue is selected
"""
c.monthly.replace(to_replace=0, value=np.inf, inplace=True)
c.monthly.head()

In [ ]:
"""
Metered Coincident Demand (MCD)

MCD is computed differently for all three meter types. 
    METERTYPE:
        CON: MCD = Normalized Usage
        DMD: MCD = min(normalized usage, billed demand)
        INT: if variance < 4% then MCD = Usage on CPDayHour
             else MCD = min(normalized usage, billed demand)
"""
from datetime import datetime
from itertools import islice
error_counts = 0
rec_count = 0
start_time = datetime.now()


for rec in c.monthly.itertuples():
    
    # Parse the record index
    prem, year = rec[0]
    
    # Parse record
    rate_class, strata, zone, stratum, tod, \
    bill_start, bill_end, usage, demand, \
    meter_type, mcd, normalized_usage, \
    meter_logic, meter_regex = rec[1:]
    
    rate_class = int(rate_class)
   
    # Service class mapping
    service_class = c.rc_map.ix[rate_class]['Map']    
    
    # Slice billcycle from temperature variants
    billcycle = c.temp_var.ix[bill_start:bill_end]
    
    # Join bill cycle with modified LoadShapeAdjustmentTable (LST)
    local_lst = pd.merge(billcycle, c.lst, 
                        left_index=True, right_index=True, # index = ObservedDate
                        on=['Max', 'DayOfWeek'])

    # Filter for Straum condition
    local_lst = local_lst[(local_lst['STRAT L BOUND'] <= float(stratum)) & 
                          (float(stratum) <= local_lst['STRAT U BOUND'])]
    
   
    # Filter for TimeOfDay meter type and Service Class Mapping
    tod_mask = local_lst['STRATA'].str.contains(meter_regex)
    sc_mask = (local_lst['SC'] == c.rc_map.ix[rate_class]['Map'])
    mask = (tod_mask == 1) & (sc_mask == 1)
    local_lst = local_lst.ix[mask]
    
    # Check to ensure proper filtering has occurred
    if local_lst.shape[0] != billcycle.shape[0]:
        error_counts += 1
        continue
    
    # Extract the kiloWatt hour columns
    kw_cols = [col for col in local_lst.columns if 'KW' in col]
    local_lst = local_lst[kw_cols]
    
    # Convert coincident peak information into usable keys
    # Compute the Customer Scaling Factor
    # Extract the Load Profile from the billing cycle
    # Compute the normalized usage
    cp_day, hr = c.cp.ix[str(year)]
    csf = usage / local_lst.values.sum()
    load_profile = local_lst.ix[cp_day]['KW' + str(hr)]
    normalized_usage = load_profile * csf
  
    
    ## MCD ##
    # if 'DMD' then MCD = min(normalized_usage, billed_demand)
    # if 'CON' then MCD = normalized_usage
    if meter_type == 'DMD':
        mcd = np.minimum(normalized_usage, demand)
    else:
        mcd = normalized_usage
        
    # Update the monthly usage values
    c.monthly.loc[(prem, year), ['NormUsage', 'MCD']
                 ] = [normalized_usage, mcd]

    
    
    rec_count += 1
    if rec_count % 1000 == 0:
        print('Current: %d Percent: %.4f Errors: %d' %(
                rec_count, rec_count / c.monthly.shape[0], error_counts))


## Timing
elapsed_time = datetime.now() - start_time

print('Total Time: %s' % elapsed_time)
print('Average Time: %f' % (elapsed_time.total_seconds() / rec_count) )

In [ ]:
c.monthly

In [ ]:
tmp = pd.merge(c.monthly.reset_index(), c.util,
        how='left',
        left_on=['Year', 'ZoneCode'],
        right_on=['Year', 'Zone'])

In [ ]:
%%time 

tmp['Factor'] = tmp['Factor'].apply(lambda x: 1.0 + x)

match_mask = tmp['MeterType_x'] == tmp['MeterType_y']
all_mask = tmp['MeterType_y'] == 'ALL'
mask = (match_mask == 1) | (all_mask == 1)

def coned_icap(g):
    #if g['ParameterId'].shape[0] != 2: # Subzone and Forecast Trueup Factors
    #    return np.nan
    #normalized_usage = g['MCD'].ix[0]
    mcd = g['MCD'].values[0]
    stf = g[g['ParameterId'] == 'SubzoneTrueupFactor']['Factor'].values[0]
    ftf = g[g['ParameterId'] == 'ForecastTrueupFactor']['Factor'].values[0]
    icap = mcd * stf * ftf

    return mcd * stf * ftf
    

In [ ]:
icap = tmp.ix[mask].groupby(by=['PremiseId', 'Year', 'RateClass', 'MeterType_x', 'MeterLogic']
                    ).apply(coned_icap).reset_index()

In [ ]:
icap[icap['PremiseId'] == '494141010200009']

In [ ]:
icap[icap['PremiseId'] == '700536121000000']

In [ ]:
historic = pd.read_sql("select * from CapacityTagHistorical where UtilityId = 'CONED'", conn)

In [ ]:
icap['Year'] = icap['Year'].apply(lambda x: x + 1)

In [ ]:
tmp = pd.merge(icap, historic, 
         left_on=['PremiseId', 'Year'],
        right_on=['PremiseID', 'CPYearID'])
tmp.rename(columns={0:'ICap'}, inplace=True)

tmp['Variance'] = (tmp['CapacityTagValue'] - tmp['ICap']) / tmp['CapacityTagValue'] * 100.

In [ ]:
tmp['Valid'] = 0
valid_idx = tmp[tmp['Variance'] <= 2.0].index
tmp.set_value(valid_idx, 'Valid', 1)

tmp.groupby(['MeterType_x', 'Year', 'RateClass', 'Valid']
           )[['ICap', 'CapacityTagValue']].count()

# Test Cases for Demand Meters

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/home/miles/Dropbox/iCAP_Project/Results/Analysis/20170419_coned_rec.csv')

In [ ]:
df = df[df.MeterType == 'DMD']

In [ ]:
df = df.dropna(axis=0, subset=['RecipeVariance']).sort_values(by='RecipeVariance')

In [ ]:
df[df.PremiseId == 644154216000007]